# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».  

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали ранее.

Построим модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нам нужно довести долю правильных ответов по крайней мере до 0.75. Проверим *accuracy* на тестовой выборке самостоятельно.

**Описание данных**  

* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Краткий план работы**
1. Знакомство с вводными данными
2. Разделение исходные данных на выборки
3. Исследование качества разных моделей
4. Проверка качества модели на тестовой выборке
5. Проверка модели на вменяемость
6. Общий вывод

### Шаг 1. Открытие файла с данными и изучение общей информации

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Предобработка данных не требуется (как следует из условий проекта), она уже была сделана ранее. Следовательно мы сразу можем перейти к следующему этапу.   
Перед нами стоит задача бинарной классификации. В качестве целевого признака выступает столбец "is_ultra". В нем отражено какой тариф использует клиент, Смарт или Ультра.

### Шаг 2. Разделим исходные данные на обучающую, валидационную и тестовую выборки

Разделим данные по принципу 60 - 20 - 20.

In [4]:
df_train, df_valid_test = train_test_split(df, train_size=0.6, test_size=0.4, random_state=42)
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=42)

print('Размер обучающей выборки', df_train.shape[0])
print('Размер валидационной выборки', df_valid.shape[0])
print('Размер тестовой выборки', df_test.shape[0])

Размер обучающей выборки 1928
Размер валидационной выборки 643
Размер тестовой выборки 643


Составим таблицы признаков и целового признака для каждого дата фрейма.

In [5]:
df_train_features = df_train.drop(['is_ultra'], axis=1)
df_train_target = df_train['is_ultra']
df_valid_features = df_valid.drop(['is_ultra'], axis=1)
df_valid_target = df_valid['is_ultra']
df_test_features = df_test.drop(['is_ultra'], axis=1)
df_test_target = df_test['is_ultra']

### Шаг 3. Исследуем качество разных моделей, меняя гиперпараметры.

Чтобы решить какой новый тариф предложить клиенту, мы должны найти взаимосвязи признаков у пользователей тарифов и понять как они работают. После этого мы сможем предположить, какой тариф оптимален для клиента. Затем нужно будет проверить наши предположения. Такой подход называется моделированием, а сами предположения и способы предсказания — моделями машинного обучения.

В процессе моделирования будем использовать следующие модели машинного обучения:
* Решающее дерево
* Случайный лес
* Логистическая регрессия  

У "Решающего дерева" и "Случайного леса" есть гиперпараметры, меняя которые можно подобрать наилучшую модель. У "Решающего дерева" это максимальная глубина дерева max_depth. У "Случайного леса" к максимальной глубине добавляется еще один гиперпараметр количество деревьев n_estimators.

Каждую модель обучим на обучающем наборе и проверим на валидационной выборке. Таким образом определим лучшую модель.

#### 3.1. Модель "Решающее дерево"

Гиперпараметр max_depth будем изменять в пределах от 2 до 19.

In [6]:
%%time

# соберем значения accuracy в список, а затем выберем из них максимальное
accuracy_tree = []

for depth in range(2, 20):
    model = DecisionTreeClassifier(random_state=42, max_depth=depth)
    model.fit(df_train_features, df_train_target)
    predictions = model.predict(df_valid_features)
    accuracy = round(accuracy_score(df_valid_target, predictions), 3)
    accuracy_tree.append(accuracy)
    
print(accuracy_tree)
print()    
print('Максимальная доля правильных ответов', max(accuracy_tree), 'при max_depth =', np.argmax(accuracy_tree)+2)
print()

[0.782, 0.792, 0.781, 0.773, 0.776, 0.781, 0.796, 0.781, 0.795, 0.785, 0.787, 0.779, 0.771, 0.768, 0.762, 0.751, 0.756, 0.75]

Максимальная доля правильных ответов 0.796 при max_depth = 8

CPU times: user 166 ms, sys: 3.21 ms, total: 169 ms
Wall time: 195 ms


Нашли лучший вариант - при глубине дерева 8 доля правильных ответов составляет **79,6%**. Как видим при глубине дерева более 14 доля правильных ответов падает, что свидетельствует о перееобучаемости модели.

#### 3.2. Модель "Случайный лес"

Гиперпараметр max_depth будем изменять в пределах от 2 до 14.
Гиперпараметр n_estimators будем изменять в пределах от 2 до 29.

In [7]:
%%time

best_accuracy = 0
for depth in range(2, 15):
    for est in range(2, 30):
        model = RandomForestClassifier(random_state=42, max_depth=depth, n_estimators=est)
        model.fit(df_train_features, df_train_target)
        predictions = model.predict(df_valid_features)
        accuracy = accuracy_score(df_valid_target, predictions)
        if accuracy > best_accuracy:
                best_accuracy = round(accuracy, 3)
                best_depth = depth
                best_est = est
print("Максимальная доля правильных ответов", best_accuracy, "при max_depth =", best_depth, "и n_estimators =", best_est)
print()

Максимальная доля правильных ответов 0.82 при max_depth = 9 и n_estimators = 24

CPU times: user 22.4 s, sys: 120 ms, total: 22.5 s
Wall time: 22.5 s


Нашли лучший вариант - при глубине дерева 9 и количестве деревьев 24, доля правильных ответов составляет **82%**.

#### 3.3. Модель "Логистическая регрессия"

In [8]:
%%time

model_lr = LogisticRegression(random_state=42, max_iter= 1000, solver = 'lbfgs')
model_lr.fit(df_train_features, df_train_target)
predictions_lr = model_lr.predict(df_valid_features)
accuracy_lr = round(accuracy_score(df_valid_target, predictions_lr), 3)

print("Доля правильных ответов", accuracy_lr)
print()

Доля правильных ответов 0.74

CPU times: user 45.2 ms, sys: 3.89 ms, total: 49 ms
Wall time: 59.1 ms


Модель логистической регресии дает **74%** правильных ответов. 

#### 3.4. Сравнение моделей

Сделаем сводную таблицу по результатам предыдущих пунктов

In [9]:
accuracy_time = {'accuracy': ['0.796', '0.82', '0.74'], 'mean_time': ['0.2', '22.0', '0.05']}
df_accuracy_time = pd.DataFrame(data = accuracy_time, index=['Решающее дерево', 'Случайный лес', 'Логистическая регрессия'])
df_accuracy_time

,accuracy,mean_time
Решающее дерево,0.796,0.2
Случайный лес,0.82,22.0
Логистическая регрессия,0.74,0.05


Самой точной оказалась модель "Случайный лес", при этом время её выполнения очень большое. Данной моделью можно воспользоваться, если заранее вычислять потенциальный тариф для клиента.  
В случае если абонент хочет выбрать для себя тариф "он-лайн", лучше воспользоваться моделью "Решающее дерево". Точность у неё не много ниже, при этом она на много быстрее.  

### Шаг 4. Проверим качество модели на тестовой выборке

#### 4.1. Модель "Решающее дерево"

In [10]:
model_tree = DecisionTreeClassifier(max_depth=8, random_state=42)
model_tree.fit(df_train_features, df_train_target)
accuracy_model_tree= round(model_tree.score(df_test_features, df_test_target), 3)
print("Точность модели:", accuracy_model_tree)

Точность модели: 0.798


79,8% правильных ответов. Практически так же как на валидационной выборке.

#### 4.2. Модель "Случайный лес"

In [11]:
model_forest = RandomForestClassifier(max_depth=9, n_estimators=24, random_state=42)
model_forest.fit(df_train_features, df_train_target)
accuracy_model_forest = round(model_forest.score(df_test_features, df_test_target), 3)
print("Точность модели:", accuracy_model_forest)

Точность модели: 0.816


81,6% правильных ответов. Не много ниже чем на валидационной выборке.

#### 4.3. Модель "Логистическая регрессия"

In [12]:
model_lr = LogisticRegression(random_state=42, max_iter= 1000, solver = 'lbfgs')
model_lr.fit(df_train_features, df_train_target)
accuracy_lr_test = round(model_lr.score(df_test_features, df_test_target), 3)
print("Точность модели:", accuracy_lr_test)

Точность модели: 0.768


76,8% правильных ответов. Точность получилась больше чем на валидационной выборке.

**Наиболее точные предсказания делает модель "Случайный лес", это видно по точности предсказания и на тестовой выборке.**

### Шаг 5. Проверим модели на вменяемость

#### 5.1. Случайное предсказание

Проверим, что наши модели работают лучше случайных предсказаний. В качестве случайных предсказаний создадим массив из нулей и единиц распределенных случайным образом.

In [13]:
random_answers = np.random.randint(0, 2, size=643)

In [14]:
accuracy_random_test = round(accuracy_score(df_test_target, random_answers), 3)
print("Точность случайных предсказаний:", accuracy_random_test)

Точность случайных предсказаний: 0.518


Любая из наших моделей дает гораздо больше правильных ответов чем случайные предсказания.

#### 5.2. Фиктивные оценки (DummyClassifier)

* Генерирация случайного прогноза

In [15]:
model_stratified = DummyClassifier(strategy="stratified" ,random_state=42)
model_stratified.fit(df_train_features, df_train_target)
accuracy_stratified = round(model_stratified.score(df_test_features, df_test_target), 3)
print("Качество прогноза :", accuracy_stratified)

Качество прогноза : 0.585


* Предсказание наиболее частой метки в обучающем наборе

In [16]:
model_most_frequent = DummyClassifier(strategy="most_frequent" ,random_state=42)
model_most_frequent.fit(df_train_features, df_train_target)
accuracy_most_frequent = round(model_most_frequent.score(df_test_features, df_test_target), 3)
print("Качество прогноза :", accuracy_most_frequent)

Качество прогноза : 0.697


* Генерация предсказания равномерно в случайном порядке

In [17]:
model_uniform = DummyClassifier(strategy="uniform" ,random_state=42)
model_uniform.fit(df_train_features, df_train_target)
accuracy_uniform = round(model_uniform.score(df_test_features, df_test_target), 3)
print("Качество прогноза :", accuracy_uniform)

Качество прогноза : 0.51


Любая из наших моделей делает более качественный прогноз.

### Шаг 6. Общий вывод

В данном проекте мы рассмотрели три модели для задачи классификации: "Решающее дерево", "Случайный лес" и "Логистическая регрессия". В каждой модели, изменяя ее гиперпараметры, мы нашли нашли лучший вариант с наибольшим количеством правильных ответов. Путем простого сравнения лучших моделей по доле правильных ответов определили, что для нашей задачи подходит модель "Случайного леса" с гиперпараметрами глубина дерева 9 и количество деревьев 24.

Сравнив нашу модель со случайными предсказаниями и оценив ее качество на тестовом наборе данных (результат составил 81,6% правильных ответов) мы подтвердили, что наша модель подходит для задачи заказчика. Таким образом можно использовать данную модель для предложения клиентам заказчика нового тарифа ("Смарт" или "Ультра").